<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/coordinates_display_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Display box coordinates resulting from object detection or image augmentation on images
---
*Last Updated 21 March 2020*   
Use this notebook to verify the coordinates resulting from either object detection or image augmentation. If results aren't as expected, data reformatting and tidying steps in [convert_bboxdims.py](https://github.com/aubricot/object_detection_for_image_cropping/blob/master/convert_bboxdims.py) or [preprocessing.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/preprocessing.ipynb) can be adjusted and results re-displayed accordingly.

__Coordinates from object detection__: Bounding boxes resulting from object detection using YOLO were exported from object_detection_for_image_cropping_yolo.ipynb, then converted to square dimensions and padded to be used as crop coordinates for EOL images using convert_bboxdims.py. Crop coordinates were exported to bird_crops_yolo_1000img_display_test.tsv and should be imported to your Google Drive for upload to this notebook.

__Coordinates from pre-processing and augmentation__: Bounding boxes resulting from pre-processing and augmentation of EOL user-generated cropping coordinates and images were tidied and exported from preprocessing.ipynb to chiroptera_crops_train_aug_fin.tsv and chiroptera_crops_test_notaug_fin.tsv and should be imported to your Google Drive for upload to this notebook. 

## Imports
---


In [0]:
# For importing data and images
import numpy as np 
import pandas as pd
import urllib

# For drawing on and displaying images
import cv2
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

# For saving images
# Un-comment out if running "Save crop dimensions displayed on images to Google Drive" below
#!pip install pillow
#!pip install scipy==1.1.0
#import scipy

##Load in crop coordinates from Google Drive
---

In [0]:
# Mount Google Drive to import your file containing coordinates (object detection bounding boxes, cropping coordinates, etc.)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Import your file containining coordinates from Google Drive
# Note: First you need to upload the file to your Google Drive, then adjust the path accordingly
#df = pd.read_csv('drive/My Drive/fall19_smithsonian_informatics/train/aves_det_crops_1000.tsv', sep='\t', header=0)
df = pd.read_csv('aves_crops_rcnn_1000img_display_test.tsv', sep='\t', header=0)
print(df.head())

## Display crop dimensions on images
---

In [0]:
# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

In [0]:
# Display crop dimensions on images
# Adjust line below to see up to 50 images displayed at a time
#for i, row in df.head(50).iterrows():
for i, row in df.iloc[50:56].iterrows():
  # Read in image 
  url = df['eolMediaURL'][i]
  img = url_to_image(url)

  # Draw box on image
  # Last values listed adjust color (255, 0, 157) and line thickness (5)
  xmin = df['xmin'][i].astype(int)
  ymin = df['ymin'][i].astype(int) + df['crop_height'][i].astype(int)
  xmax = df['xmin'][i].astype(int) + df['crop_width'][i].astype(int)
  ymax = df['ymin'][i].astype(int)
  image_wbox = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 157), 5) 
  
  # Plot and show cropping boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(image_wbox)
  # Display image URL above image to facilitate troubleshooting/fine-tuning of data reformatting and tidying steps in convert_bboxdims.py or preprocessing.ipynb
  plt.title('{} xmin:{}, ymin: {}, xmax:{}, ymax:{}'.format(url, xmin, ymin, xmax, ymax))

## Save crop dimensions displayed on images to Google Drive
---
Useful if want to share results with someone remotely. Saves images with detection boxes to Google Drive.

In [0]:
# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

In [29]:
# For saving images
from scipy import misc

# Path to folder for exporting images with bounding boxes
pathbase = '/content/drive/My Drive/fall19_smithsonian_informatics/train/out/new_bboxdims_colab/'

# Display crop dimensions on images
# Adjust line below to see up to 50 images displayed at a time
#for i, row in df.head(50).iterrows():
for i, row in df.iloc[50:56].iterrows():
  # Make output path
  path = pathbase + str(df.dataObjectVersionID[i]) + '.jpg'

  # Read in image 
  url = df['eolMediaURL'][i]
  img = url_to_image(url)

  # Draw box on image
  # Last values listed adjust color (255, 0, 157) and line thickness (5)
  xmin = df['xmin'][i].astype(int)
  ymax = df['ymin'][i].astype(int) + df['crop_height'][i].astype(int)
  xmax = df['xmin'][i].astype(int) + df['crop_width'][i].astype(int)
  ymin = df['ymin'][i].astype(int)
  image_wbox = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 157), 5) 

  # Export image to Google Drive
  misc.imsave(path, image_wbox)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
